# Supporting Cyclic Garbage Collection

Python's support for detecting and collecting garbage which involves
circular references requires support from object types which are
"containers" for other objects which may also be containers. Types which
do not store references to other objects, or which only store references
to atomic types (such as numbers or strings), do not need to provide any
explicit support for garbage collection.

To create a container type, the :c`~PyTypeObject.tp_flags` field of the
type object must include the `Py_TPFLAGS_HAVE_GC` and provide an
implementation of the :c`~PyTypeObject.tp_traverse` handler. If
instances of the type are mutable, a :c`~PyTypeObject.tp_clear`
implementation must also be provided.

Py_TPFLAGS_HAVE_GC

Objects with a type with this flag set must conform with the rules
documented here. For convenience these objects will be referred to as
container objects.

Constructors for container types must conform to two rules:

1.  The memory for the object must be allocated using
    :c`PyObject_GC_New` or :c`PyObject_GC_NewVar`.
2.  Once all the fields which may contain references to other containers
    are initialized, it must call :c`PyObject_GC_Track`.

Similarly, the deallocator for the object must conform to a similar pair
of rules:

1.  Before fields which refer to other containers are invalidated,
    :c`PyObject_GC_UnTrack` must be called.

2.  The object's memory must be deallocated using :c`PyObject_GC_Del`.

    Warning

    If a type adds the Py_TPFLAGS_HAVE_GC, then it *must* implement at
    least a :c`~PyTypeObject.tp_traverse` handler or explicitly use one
    from its subclass or subclasses.

    When calling :c`PyType_Ready` or some of the APIs that indirectly
    call it like :c`PyType_FromSpecWithBases` or :c`PyType_FromSpec` the
    interpreter will automatically populate the
    :c`~PyTypeObject.tp_flags`, :c`~PyTypeObject.tp_traverse` and
    :c`~PyTypeObject.tp_clear` fields if the type inherits from a class
    that implements the garbage collector protocol and the child class
    does *not* include the `Py_TPFLAGS_HAVE_GC` flag.

> Analogous to :c`PyObject_New` but for container objects with the
> `Py_TPFLAGS_HAVE_GC` flag set.

> Analogous to :c`PyObject_NewVar` but for container objects with the
> `Py_TPFLAGS_HAVE_GC` flag set.

> Resize an object allocated by :c`PyObject_NewVar`. Returns the resized
> object or `NULL` on failure. *op* must not be tracked by the collector
> yet.

> Adds the object *op* to the set of container objects tracked by the
> collector. The collector can run at unexpected times so objects must
> be valid while being tracked. This should be called once all the
> fields followed by the :c`~PyTypeObject.tp_traverse` handler become
> valid, usually near the end of the constructor.

> Returns non-zero if the object implements the garbage collector
> protocol, otherwise returns 0.
>
> The object cannot be tracked by the garbage collector if this function
> returns 0.

> Returns 1 if the object type of *op* implements the GC protocol and
> *op* is being currently tracked by the garbage collector and 0
> otherwise.
>
> This is analogous to the Python function `gc.is_tracked`.
>
> 3.9

> Returns 1 if the object type of *op* implements the GC protocol and
> *op* has been already finalized by the garbage collector and 0
> otherwise.
>
> This is analogous to the Python function `gc.is_finalized`.
>
> 3.9

> Releases memory allocated to an object using :c`PyObject_GC_New` or
> :c`PyObject_GC_NewVar`.

> Remove the object *op* from the set of container objects tracked by
> the collector. Note that :c`PyObject_GC_Track` can be called again on
> this object to add it back to the set of tracked objects. The
> deallocator (:c`~PyTypeObject.tp_dealloc` handler) should call this
> for the object before any of the fields used by the
> :c`~PyTypeObject.tp_traverse` handler become invalid.

3.8

The :c`_PyObject_GC_TRACK` and :c`_PyObject_GC_UNTRACK` macros have been
removed from the public C API.

The :c`~PyTypeObject.tp_traverse` handler accepts a function parameter
of this type:

> Type of the visitor function passed to the
> :c`~PyTypeObject.tp_traverse` handler. The function should be called
> with an object to traverse as *object* and the third parameter to the
> :c`~PyTypeObject.tp_traverse` handler as *arg*. The Python core uses
> several visitor functions to implement cyclic garbage detection; it's
> not expected that users will need to write their own visitor
> functions.

The :c`~PyTypeObject.tp_traverse` handler must have the following type:

> Traversal function for a container object. Implementations must call
> the *visit* function for each object directly contained by *self*,
> with the parameters to *visit* being the contained object and the
> *arg* value passed to the handler. The *visit* function must not be
> called with a `NULL` object argument. If *visit* returns a non-zero
> value that value should be returned immediately.

To simplify writing :c`~PyTypeObject.tp_traverse` handlers, a
:c`Py_VISIT` macro is provided. In order to use this macro, the
:c`~PyTypeObject.tp_traverse` implementation must name its arguments
exactly *visit* and *arg*:

> If *o* is not `NULL`, call the *visit* callback, with arguments *o*
> and *arg*. If *visit* returns a non-zero value, then return it. Using
> this macro, :c`~PyTypeObject.tp_traverse` handlers look like:
>
> ``` c
> static int
> my_traverse(Noddy *self, visitproc visit, void *arg)
> {
>     Py_VISIT(self->foo);
>     Py_VISIT(self->bar);
>     return 0;
> }
> ```

The :c`~PyTypeObject.tp_clear` handler must be of the :c`inquiry` type,
or `NULL` if the object is immutable.

> Drop references that may have created reference cycles. Immutable
> objects do not have to define this method since they can never
> directly create reference cycles. Note that the object must still be
> valid after calling this method (don't just call :c`Py_DECREF` on a
> reference). The collector will call this method if it detects that
> this object is involved in a reference cycle.

## Controlling the Garbage Collector State

The C-API provides the following functions for controlling garbage
collection runs.

> Perform a full garbage collection, if the garbage collector is
> enabled. (Note that `gc.collect` runs it unconditionally.)
>
> Returns the number of collected + unreachable objects which cannot be
> collected. If the garbage collector is disabled or already collecting,
> returns `0` immediately. Errors during garbage collection are passed
> to `sys.unraisablehook`. This function does not raise exceptions.

> Enable the garbage collector: similar to `gc.enable`. Returns the
> previous state, 0 for disabled and 1 for enabled.
>
> 3.10

> Disable the garbage collector: similar to `gc.disable`. Returns the
> previous state, 0 for disabled and 1 for enabled.
>
> 3.10

> Query the state of the garbage collector: similar to `gc.isenabled`.
> Returns the current state, 0 for disabled and 1 for enabled.
>
> 3.10